In [3]:
import pandas as pd
import crew_optimizer
import openai
import dotenv
import json

In [16]:
assignment = pd.read_json("assignments/assignment.json")
assignment = assignment['assignment'][4:254]
assignment

D1_F026    {'captain': 'C26', 'first_officer': 'FO20', 'd...
D1_F043    {'captain': 'C44', 'first_officer': 'FO35', 'd...
D1_F005    {'captain': 'C34', 'first_officer': 'FO46', 'd...
D1_F023    {'captain': 'C6', 'first_officer': 'FO28', 'de...
D1_F031    {'captain': 'C32', 'first_officer': 'FO32', 'd...
                                 ...                        
D5_F042    {'captain': 'C31', 'first_officer': 'FO19', 'd...
D5_F003    {'captain': 'C36', 'first_officer': 'FO2', 'de...
D5_F001    {'captain': 'C35', 'first_officer': 'FO21', 'd...
D5_F021    {'captain': 'C45', 'first_officer': 'FO0', 'de...
D5_F023    {'captain': 'C21', 'first_officer': 'FO14', 'd...
Name: assignment, Length: 250, dtype: object